In [1]:
import numpy as np
import pandas as pd
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
# import pydot
# from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import os
from glob import glob
import shutil
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
path = "data/"

In [3]:
dir_names = [w for w in os.listdir(path + 'train/') if os.path.isdir(os.path.join(path + 'train/', w))]
dir_names

['Parasitized', 'Uninfected']

In [4]:
os.mkdir(path + 'valid/')

In [5]:
# Create sub folder for categories in valid folder
for i in dir_names:
    os.mkdir(path + 'valid/' + i)

In [6]:
# Moving images in folder
for d in dir_names:
    g = glob(path + 'train/' + d + '/' + '*.png')
    shuf = np.random.permutation(g)
    
    for i in range(int(len(g) / 5)): shutil.move(shuf[i], path + 'valid/' + d +'/')

In [7]:
def identity_block(X, f, filters, stage, block):
    
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [8]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [9]:
def ResNet50(input_shape=(110, 110, 3), classes=2):

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(25, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [10]:
model = ResNet50(input_shape = (110, 110, 3), classes = 2)

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
batch_size = 100
datagen = image.ImageDataGenerator()
trn_batches = datagen.flow_from_directory('data/train/', target_size = (110, 110), 
                                          batch_size = batch_size, class_mode = 'categorical')
val_batches = datagen.flow_from_directory('data/valid/', target_size = (110, 110), 
                                          batch_size = batch_size, class_mode = 'categorical')

Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.


In [13]:
filepath = "../weights/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

In [14]:
model.fit_generator(trn_batches, steps_per_epoch = trn_batches.n // batch_size, epochs = 50, 
                    validation_data = val_batches, validation_steps = val_batches.n // batch_size, 
                    callbacks = callbacks_list)

Epoch 1/50
220/220 [==============================] - 116s 526ms/step - loss: 0.9436 - acc: 0.6606 - val_loss: 0.3537 - val_acc: 0.8889

Epoch 00001: val_acc improved from -inf to 0.88891, saving model to ../weights/weights-improvement-01-0.89.hdf5
Epoch 2/50
220/220 [==============================] - 104s 474ms/step - loss: 0.2563 - acc: 0.9224 - val_loss: 0.1719 - val_acc: 0.9398

Epoch 00002: val_acc improved from 0.88891 to 0.93982, saving model to ../weights/weights-improvement-02-0.94.hdf5
Epoch 3/50
220/220 [==============================] - 104s 473ms/step - loss: 0.1485 - acc: 0.9504 - val_loss: 0.1510 - val_acc: 0.9456

Epoch 00003: val_acc improved from 0.93982 to 0.94564, saving model to ../weights/weights-improvement-03-0.95.hdf5
Epoch 4/50
220/220 [==============================] - 104s 474ms/step - loss: 0.1829 - acc: 0.9413 - val_loss: 0.1504 - val_acc: 0.9467

Epoch 00004: val_acc improved from 0.94564 to 0.94673, saving model to ../weights/weights-improvement-04-0.95.


Epoch 00041: val_acc did not improve from 0.96000
Epoch 42/50
220/220 [==============================] - 103s 470ms/step - loss: 0.0171 - acc: 0.9936 - val_loss: 0.2191 - val_acc: 0.9560

Epoch 00042: val_acc did not improve from 0.96000
Epoch 43/50
220/220 [==============================] - 104s 471ms/step - loss: 0.0126 - acc: 0.9952 - val_loss: 0.2774 - val_acc: 0.9556

Epoch 00043: val_acc did not improve from 0.96000
Epoch 44/50
220/220 [==============================] - 104s 472ms/step - loss: 0.0131 - acc: 0.9947 - val_loss: 0.2529 - val_acc: 0.9571

Epoch 00044: val_acc did not improve from 0.96000
Epoch 45/50
220/220 [==============================] - 104s 471ms/step - loss: 0.0098 - acc: 0.9960 - val_loss: 0.2948 - val_acc: 0.9584

Epoch 00045: val_acc did not improve from 0.96000
Epoch 46/50
220/220 [==============================] - 104s 471ms/step - loss: 0.0145 - acc: 0.9947 - val_loss: 0.2257 - val_acc: 0.9504

Epoch 00046: val_acc did not improve from 0.96000
Epoch 47/

# Prediction

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
# Read Images 
img = mpimg.imread("data/train/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_162.png") 

In [ ]:
img.shape

In [ ]:
img = np.resize(img, (110, 110, 3))

In [ ]:
image = np.expand_dims(img, axis=0)
image = preprocess_input(image)

In [ ]:
image.shape

In [ ]:
model.predict_on_batch(image)